# 02 — Retrieval Planning

Test the Retrieval Planner agent — the first ReAct agent in the pipeline.

**What we're testing:**
1. Claim characteristic detection (regex/keyword tool)
2. Guideline coverage checking
3. Rule-based retrieval planning (no API key needed)
4. Full ReAct planning with Claude (needs ANTHROPIC_API_KEY)
5. End-to-end: Decomposer → Retrieval Planner

In [1]:
import sys
sys.path.insert(0, '..')

import warnings
warnings.filterwarnings('ignore')

In [2]:
CLAIMS = [
    "Intermittent fasting reverses Type 2 diabetes",
    "Vitamin D prevents COVID-19 infection",
    "Metformin reduces cancer risk in diabetic patients",
    "Turmeric is as effective as ibuprofen for arthritis pain",
    "Drinking 8 glasses of water a day prevents kidney stones",
    "Aspirin interacts with warfarin and increases bleeding risk",
    "Patients with hypertension should take ACE inhibitors as first-line treatment",
    "Omega-3 supplements reduce triglycerides by 30%",
]

---
## 1. Claim Characteristic Detection

The `analyze_claim_characteristics` tool uses regex and keywords to detect features of each claim — no LLM needed.

In [ ]:
from systems.s4_langgraph.agents.retrieval_planner import _tool_analyze_claim
from src.medical_nlp.medical_ner import extract_entities

print("Claim Characteristic Detection")
print("=" * 80)

for claim in CLAIMS:
    entities = extract_entities(claim)
    entities_dict = {
        "drugs": entities.drugs,
        "conditions": entities.conditions,
    }
    result = _tool_analyze_claim(claim, entities_dict)

    # Show only True flags
    flags = [k for k in [
        "has_drugs", "has_drug_interaction", "has_numbers",
        "asks_recommendation", "asks_effectiveness",
        "has_comparison", "asks_safety",
    ] if result.get(k)]

    print(f"\nClaim: {claim}")
    print(f"  Entities: drugs={entities.drugs}, conditions={entities.conditions}")
    print(f"  Flags:    {flags or ['(none)']}")
    if result["number_matches"]:
        print(f"  Numbers:  {result['number_matches']}")

---
## 2. Guideline Coverage Check

Check which claims have matching WHO/NIH/MOH guideline topics.

In [ ]:
from systems.s4_langgraph.agents.retrieval_planner import _tool_check_guidelines

print("Guideline Coverage Check")
print("=" * 80)

for claim in CLAIMS:
    entities = extract_entities(claim)
    entities_dict = {
        "drugs": entities.drugs,
        "conditions": entities.conditions,
    }
    result = _tool_check_guidelines(claim, entities_dict)

    print(f"\nClaim: {claim}")
    if result["has_guideline_coverage"]:
        for source, topics in result["matching_sources"].items():
            print(f"  {source.upper()}: {', '.join(topics)}")
    else:
        print("  No guideline coverage")

---
## 3. Rule-Based Retrieval Planning (no API key)

Deterministic planning using keyword matching. This is the fallback when no Anthropic API key is available.

In [ ]:
from systems.s4_langgraph.agents.retrieval_planner import _plan_rule_based, DEFAULT_METHODS
from src.models import SubClaim, PICO
from src.medical_nlp.pico_extractor import extract_pico_rule_based

print("Rule-Based Retrieval Plans")
print("=" * 80)

for claim in CLAIMS:
    entities = extract_entities(claim)
    entities_dict = {
        "drugs": entities.drugs,
        "conditions": entities.conditions,
        "genes": entities.genes,
        "organisms": entities.organisms,
        "procedures": entities.procedures,
        "anatomical": entities.anatomical,
    }
    pico = extract_pico_rule_based(claim, entities)

    state = {
        "claim": claim,
        "sub_claims": [SubClaim(id="sc-1", text=claim, pico=pico)],
        "entities": entities_dict,
        "pico": pico,
    }
    plan = _plan_rule_based(state)

    # Highlight methods beyond the defaults
    methods = plan["sc-1"]
    extra = [m for m in methods if m not in DEFAULT_METHODS]

    print(f"\nClaim: {claim}")
    print(f"  Methods:  {methods}")
    if extra:
        print(f"  Extra:    {extra} (beyond defaults)")

---
## 4. Full Pipeline: Decomposer → Retrieval Planner

Run the decomposer first to get sub-claims, then feed into the retrieval planner.

This uses `run_retrieval_planner` which will use ReAct (if API key is set) or rule-based fallback.

In [ ]:
import asyncio
from src.functions.decomposer import run_decomposer
from systems.s4_langgraph.agents.retrieval_planner import run_retrieval_planner

# Build initial state
claim = "Turmeric is as effective as ibuprofen for arthritis pain"

initial_state = {
    "claim": claim,
    "pico": None,
    "sub_claims": [],
    "entities": {},
    "retrieval_plan": {},
    "evidence": [],
    "extracted_figures": [],
    "evidence_quality": {},
    "verdict": "",
    "confidence": 0.0,
    "explanation": "",
    "safety_flags": [],
    "is_dangerous": False,
    "agent_trace": [],
    "total_cost_usd": 0.0,
    "total_duration_seconds": 0.0,
}

# Step 1: Decompose
state = await run_decomposer(initial_state)

print(f"Claim: {claim}")
print(f"\nPICO: P={state['pico'].population}, I={state['pico'].intervention}, "
      f"C={state['pico'].comparison}, O={state['pico'].outcome}")
print(f"\nSub-claims ({len(state['sub_claims'])}):\n")
for sc in state["sub_claims"]:
    print(f"  {sc.id}: {sc.text}")

In [7]:
# Step 2: Retrieval Planning
state = await run_retrieval_planner(state)

print("Retrieval Plan")
print("=" * 60)
for sc_id, methods in state["retrieval_plan"].items():
    sc_text = next((sc.text for sc in state["sub_claims"] if sc.id == sc_id), "?")
    print(f"\n  {sc_id}: \"{sc_text}\"")
    print(f"    Methods: {methods}")

# Show trace
print("\n" + "=" * 60)
print("Agent Traces:")
for trace in state["agent_trace"]:
    print(f"  [{trace.agent}] {trace.node_type} | "
          f"{trace.duration_seconds}s | ${trace.cost_usd:.4f} | "
          f"steps={trace.reasoning_steps}")
print(f"\nTotal cost: ${state['total_cost_usd']:.4f}")
print(f"Total time: {state['total_duration_seconds']:.2f}s")

Retrieval Plan

  sc-1: "Turmeric is as effective as ibuprofen for arthritis pain"
    Methods: ['pubmed_api', 'semantic_scholar', 'cross_encoder', 'cochrane_api', 'clinical_trials']

Agent Traces:
  [decomposer] function | 4.0s | $0.0024 | steps=0
  [retrieval_planner] agent | 10.11s | $0.0239 | steps=3

Total cost: $0.0263
Total time: 14.10s


---
## 5. Compare: All Claims Through the Pipeline

Run all test claims through decomposer → planner and compare the retrieval plans.

In [ ]:
import pandas as pd
from systems.s4_langgraph.agents.retrieval_planner import VALID_METHODS

rows = []
matrix_rows = []

for claim in CLAIMS:
    state = {
        "claim": claim,
        "pico": None, "sub_claims": [], "entities": {},
        "retrieval_plan": {}, "evidence": [], "extracted_figures": [],
        "evidence_quality": {}, "verdict": "", "confidence": 0.0,
        "explanation": "", "safety_flags": [], "is_dangerous": False,
        "agent_trace": [], "total_cost_usd": 0.0,
        "total_duration_seconds": 0.0,
    }

    state = await run_decomposer(state)
    state = await run_retrieval_planner(state)

    for sc_id, methods in state["retrieval_plan"].items():
        sc_text = next((sc.text for sc in state["sub_claims"] if sc.id == sc_id), "?")
        rows.append({
            "claim": claim[:50],
            "sub_claim": f"{sc_id}: {sc_text[:50]}",
            "methods": ", ".join(methods),
            "n_methods": len(methods),
            "cost": f"${state['total_cost_usd']:.4f}",
        })

        # Build matrix row: 1 if method assigned, 0 if skipped
        method_set = set(methods)
        matrix_row = {"claim": claim[:45]}
        for m in sorted(VALID_METHODS):
            matrix_row[m] = "Y" if m in method_set else "-"
        matrix_row["total"] = len(methods)
        matrix_rows.append(matrix_row)

df = pd.DataFrame(rows)
print(df.to_string(index=False))

---
## 6. Method × Claim Matrix

Shows exactly which methods are assigned (Y) and skipped (-) per claim. This is the key diagnostic — if every column is `Y`, the planner isn't discriminating.

In [9]:
matrix_df = pd.DataFrame(matrix_rows)

# Reorder columns: claim first, then defaults, then optional, then total
default_cols = ["pubmed_api", "semantic_scholar", "cross_encoder"]
optional_cols = [m for m in sorted(VALID_METHODS) if m not in default_cols]

matrix_df = matrix_df[["claim"] + default_cols + optional_cols + ["total"]]

print("Method Assignment Matrix  (Y = assigned, - = skipped)")
print("=" * 120)
print(matrix_df.to_string(index=False))

# Summary: how often is each optional method used?
print("\n\nOptional Method Usage Frequency:")
print("-" * 40)
for col in optional_cols:
    count = (matrix_df[col] == "Y").sum()
    pct = count / len(matrix_df) * 100
    print(f"  {col:20s}  {count}/{len(matrix_df)}  ({pct:.0f}%)")

Method Assignment Matrix  (Y = assigned, - = skipped)
                                        claim pubmed_api semantic_scholar cross_encoder clinical_trials cochrane_api deep_search drugbank_api guideline_store  total
Intermittent fasting reverses Type 2 diabetes          Y                Y             Y               -            -           -            -               -      3
        Vitamin D prevents COVID-19 infection          Y                Y             Y               Y            Y           -            -               -      5
Metformin reduces cancer risk in diabetic pat          Y                Y             Y               Y            -           -            -               -      4
Turmeric is as effective as ibuprofen for art          Y                Y             Y               Y            Y           -            -               -      5
Drinking 8 glasses of water a day prevents ki          Y                Y             Y               -            -     

---
## 7. Discrimination Check

**What to look for:**
- `drugbank_api` should ONLY fire for the aspirin/warfarin interaction claim
- `guideline_store` should fire for hypertension/diabetes/cancer — NOT for "8 glasses of water"
- `deep_search` should fire for the "30%" claim — NOT for qualitative claims
- `clinical_trials` + `cochrane_api` should fire for treatment effectiveness, NOT for drug interactions

If an optional method fires for >75% of claims, it's not discriminating well.

In [10]:
# Automated discrimination checks
print("Discrimination Report")
print("=" * 60)

checks = []

# 1. drugbank_api should only fire for interaction claims
drugbank_claims = matrix_df[matrix_df["drugbank_api"] == "Y"]["claim"].tolist()
expected_drugbank = ["Aspirin interacts with warfarin"]
checks.append({
    "method": "drugbank_api",
    "rule": "only drug interaction claims",
    "fired_for": drugbank_claims,
    "pass": all("interact" in c.lower() for c in drugbank_claims) and len(drugbank_claims) > 0,
})

# 2. deep_search should only fire for quantitative claims
deep_claims = matrix_df[matrix_df["deep_search"] == "Y"]["claim"].tolist()
checks.append({
    "method": "deep_search",
    "rule": "only quantitative claims (has numbers/percentages)",
    "fired_for": deep_claims,
    "pass": len(deep_claims) <= 3,  # should be selective
})

# 3. guideline_store should NOT fire for everything
guide_claims = matrix_df[matrix_df["guideline_store"] == "Y"]["claim"].tolist()
guide_pct = len(guide_claims) / len(matrix_df) * 100
checks.append({
    "method": "guideline_store",
    "rule": "should not fire for >75% of claims",
    "fired_for": guide_claims,
    "pass": guide_pct <= 75,
})

# 4. At least one claim should get only the 3 defaults (nothing extra)
defaults_only = matrix_df[matrix_df["total"] == 3]
checks.append({
    "method": "(baseline)",
    "rule": "at least 1 claim gets only 3 defaults",
    "fired_for": defaults_only["claim"].tolist(),
    "pass": len(defaults_only) > 0,
})

for c in checks:
    status = "PASS" if c["pass"] else "FAIL"
    print(f"\n  [{status}] {c['method']}: {c['rule']}")
    if c["fired_for"]:
        for cl in c["fired_for"]:
            print(f"         -> {cl}")
    else:
        print(f"         -> (none)")

n_pass = sum(1 for c in checks if c["pass"])
print(f"\n{'=' * 60}")
print(f"Result: {n_pass}/{len(checks)} checks passed")

Discrimination Report

  [PASS] drugbank_api: only drug interaction claims
         -> Aspirin interacts with warfarin and increases

  [PASS] deep_search: only quantitative claims (has numbers/percentages)
         -> Omega-3 supplements reduce triglycerides by 3

  [PASS] guideline_store: should not fire for >75% of claims
         -> Patients with hypertension should take ACE in

  [PASS] (baseline): at least 1 claim gets only 3 defaults
         -> Intermittent fasting reverses Type 2 diabetes
         -> Drinking 8 glasses of water a day prevents ki

Result: 4/4 checks passed


In [11]:
# Save v1 results for before/after comparison
v1_matrix_df = matrix_df.copy()
v1_checks_passed = n_pass
v1_total_checks = len(checks)
print(f"Saved v1 results: {v1_checks_passed}/{v1_total_checks} checks passed")

Saved v1 results: 4/4 checks passed


---
## 8. Re-run with Tightened System Prompt (v2)

The system prompt was updated to:
- Tell the agent to be **selective, not comprehensive** — use the MINIMUM set
- Restrict `cochrane_api` + `clinical_trials` to explicit treatment/efficacy claims only
- Restrict `drugbank_api` to explicit drug-drug interaction/combination claims only
- Tell the agent: if no special flags, assign ONLY the 3 baseline methods

**Restart the kernel first** to pick up the updated prompt, then run this section.

In [ ]:
# Reload the module to pick up the new system prompt
import importlib
import systems.s4_langgraph.agents.retrieval_planner as rp_module
importlib.reload(rp_module)
from systems.s4_langgraph.agents.retrieval_planner import run_retrieval_planner, VALID_METHODS, DEFAULT_METHODS

v2_rows = []
v2_matrix_rows = []

for claim in CLAIMS:
    state = {
        "claim": claim,
        "pico": None, "sub_claims": [], "entities": {},
        "retrieval_plan": {}, "evidence": [], "extracted_figures": [],
        "evidence_quality": {}, "verdict": "", "confidence": 0.0,
        "explanation": "", "safety_flags": [], "is_dangerous": False,
        "agent_trace": [], "total_cost_usd": 0.0,
        "total_duration_seconds": 0.0,
    }

    state = await run_decomposer(state)
    state = await run_retrieval_planner(state)

    for sc_id, methods in state["retrieval_plan"].items():
        sc_text = next((sc.text for sc in state["sub_claims"] if sc.id == sc_id), "?")
        v2_rows.append({
            "claim": claim[:50],
            "sub_claim": f"{sc_id}: {sc_text[:50]}",
            "methods": ", ".join(methods),
            "n_methods": len(methods),
        })

        method_set = set(methods)
        matrix_row = {"claim": claim[:45]}
        for m in sorted(VALID_METHODS):
            matrix_row[m] = "Y" if m in method_set else "-"
        matrix_row["total"] = len(methods)
        v2_matrix_rows.append(matrix_row)

v2_matrix_df = pd.DataFrame(v2_matrix_rows)
default_cols = ["pubmed_api", "semantic_scholar", "cross_encoder"]
optional_cols = [m for m in sorted(VALID_METHODS) if m not in default_cols]
v2_matrix_df = v2_matrix_df[["claim"] + default_cols + optional_cols + ["total"]]

print("v2 Method Assignment Matrix  (Y = assigned, - = skipped)")
print("=" * 120)
print(v2_matrix_df.to_string(index=False))

print("\n\nv2 Optional Method Usage Frequency:")
print("-" * 40)
for col in optional_cols:
    count = (v2_matrix_df[col] == "Y").sum()
    pct = count / len(v2_matrix_df) * 100
    print(f"  {col:20s}  {count}/{len(v2_matrix_df)}  ({pct:.0f}%)")

In [13]:
# v2 discrimination checks (same checks as before)
print("v2 Discrimination Report")
print("=" * 60)

v2_checks = []

drugbank_claims = v2_matrix_df[v2_matrix_df["drugbank_api"] == "Y"]["claim"].tolist()
v2_checks.append({
    "method": "drugbank_api",
    "rule": "only drug interaction claims",
    "fired_for": drugbank_claims,
    "pass": all("interact" in c.lower() for c in drugbank_claims) and len(drugbank_claims) > 0,
})

deep_claims = v2_matrix_df[v2_matrix_df["deep_search"] == "Y"]["claim"].tolist()
v2_checks.append({
    "method": "deep_search",
    "rule": "only quantitative claims (has numbers/percentages)",
    "fired_for": deep_claims,
    "pass": len(deep_claims) <= 3,
})

guide_claims = v2_matrix_df[v2_matrix_df["guideline_store"] == "Y"]["claim"].tolist()
guide_pct = len(guide_claims) / len(v2_matrix_df) * 100
v2_checks.append({
    "method": "guideline_store",
    "rule": "should not fire for >75% of claims",
    "fired_for": guide_claims,
    "pass": guide_pct <= 75,
})

defaults_only = v2_matrix_df[v2_matrix_df["total"] == 3]
v2_checks.append({
    "method": "(baseline)",
    "rule": "at least 1 claim gets only 3 defaults",
    "fired_for": defaults_only["claim"].tolist(),
    "pass": len(defaults_only) > 0,
})

for c in v2_checks:
    status = "PASS" if c["pass"] else "FAIL"
    print(f"\n  [{status}] {c['method']}: {c['rule']}")
    if c["fired_for"]:
        for cl in c["fired_for"]:
            print(f"         -> {cl}")
    else:
        print(f"         -> (none)")

v2_n_pass = sum(1 for c in v2_checks if c["pass"])
print(f"\n{'=' * 60}")
print(f"Result: {v2_n_pass}/{len(v2_checks)} checks passed")

v2 Discrimination Report

  [PASS] drugbank_api: only drug interaction claims
         -> Aspirin interacts with warfarin and increases

  [PASS] deep_search: only quantitative claims (has numbers/percentages)
         -> Omega-3 supplements reduce triglycerides by 3

  [PASS] guideline_store: should not fire for >75% of claims
         -> Patients with hypertension should take ACE in

  [PASS] (baseline): at least 1 claim gets only 3 defaults
         -> Intermittent fasting reverses Type 2 diabetes
         -> Vitamin D prevents COVID-19 infection
         -> Drinking 8 glasses of water a day prevents ki

Result: 4/4 checks passed


---
## 9. Before / After Comparison

In [14]:
# Side-by-side frequency comparison
print("Optional Method Usage: v1 vs v2")
print("=" * 60)
print(f"{'method':20s}  {'v1':>8s}  {'v2':>8s}  {'change':>8s}")
print("-" * 60)

for col in optional_cols:
    v1_count = (v1_matrix_df[col] == "Y").sum()
    v2_count = (v2_matrix_df[col] == "Y").sum()
    v1_pct = v1_count / len(v1_matrix_df) * 100
    v2_pct = v2_count / len(v2_matrix_df) * 100
    delta = v2_pct - v1_pct
    arrow = "<<" if delta < -10 else (">>" if delta > 10 else "  ")
    print(f"  {col:20s}  {v1_pct:5.0f}%    {v2_pct:5.0f}%   {delta:+5.0f}% {arrow}")

print("-" * 60)
v1_avg = v1_matrix_df["total"].mean()
v2_avg = v2_matrix_df["total"].mean()
print(f"  {'avg methods/claim':20s}  {v1_avg:5.1f}    {v2_avg:5.1f}   {v2_avg - v1_avg:+5.1f}")

print(f"\n  Discrimination checks:  v1={v1_checks_passed}/{v1_total_checks}  →  v2={v2_n_pass}/{len(v2_checks)}")

# Per-claim total comparison
print("\n\nPer-Claim Method Count: v1 vs v2")
print("=" * 70)
print(f"{'claim':45s}  {'v1':>4s}  {'v2':>4s}  {'diff':>5s}")
print("-" * 70)
for i in range(min(len(v1_matrix_df), len(v2_matrix_df))):
    claim = v1_matrix_df.iloc[i]["claim"]
    t1 = v1_matrix_df.iloc[i]["total"]
    t2 = v2_matrix_df.iloc[i]["total"]
    diff = t2 - t1
    marker = " *" if diff != 0 else ""
    print(f"  {claim:45s}  {t1:>3}   {t2:>3}   {diff:>+3}{marker}")

Optional Method Usage: v1 vs v2
method                      v1        v2    change
------------------------------------------------------------
  clinical_trials          75%       50%     -25% <<
  cochrane_api             50%       25%     -25% <<
  deep_search              12%       12%      +0%   
  drugbank_api             12%       12%      +0%   
  guideline_store          12%       12%      +0%   
------------------------------------------------------------
  avg methods/claim       4.6      4.1    -0.5

  Discrimination checks:  v1=4/4  →  v2=4/4


Per-Claim Method Count: v1 vs v2
claim                                            v1    v2   diff
----------------------------------------------------------------------
  Intermittent fasting reverses Type 2 diabetes    3     3    +0
  Vitamin D prevents COVID-19 infection            5     3    -2 *
  Metformin reduces cancer risk in diabetic pat    4     4    +0
  Turmeric is as effective as ibuprofen for art    5     5    +0
  Dri

---
## 10. PICO Extraction Evaluation

Evaluate rule-based and LLM PICO extraction against 30 hand-labeled claims.

**Scoring:** Token overlap (Jaccard similarity) between extracted and expected values.
A score > 0.5 counts as a soft match. This is more forgiving than exact match — 
"diabetic patients" vs "patients with diabetes" both get credit.

In [15]:
import json
import importlib
import snowballstemmer

import src.medical_nlp.pico_extractor as pico_module
importlib.reload(pico_module)
from src.medical_nlp.pico_extractor import extract_pico_rule_based, extract_pico_with_llm

from src.medical_nlp.medical_ner import extract_entities

# Load ground truth
with open("../data/claims/pico_ground_truth.json") as f:
    pico_gt = json.load(f)

print(f"Loaded {len(pico_gt)} claims with PICO ground truth")
print(f"Sample: {pico_gt[0]['claim']}")
print(f"  Expected: {pico_gt[0]['pico']}")

_stemmer = snowballstemmer.stemmer("english")

PICO_ELEMENTS = ["population", "intervention", "comparison", "outcome"]


def tokenize(text):
    """Lowercase, split into words, and stem each token."""
    if not text:
        return set()
    return set(_stemmer.stemWords(text.lower().split()))


def jaccard(a, b):
    """Token-level Jaccard similarity (with stemming)."""
    ta, tb = tokenize(a), tokenize(b)
    if not ta and not tb:
        return 1.0
    if not ta or not tb:
        return 0.0
    return len(ta & tb) / len(ta | tb)


def token_f1(a, b):
    """Token-level F1 score (with stemming).
    
    Like SQuAD scoring: precision = overlap/extracted, recall = overlap/expected.
    More informative than Jaccard — tells you if the model extracts too much vs too little.
    """
    ta, tb = tokenize(a), tokenize(b)
    if not ta and not tb:
        return 1.0
    if not ta or not tb:
        return 0.0
    common = ta & tb
    if not common:
        return 0.0
    precision = len(common) / len(ta)
    recall = len(common) / len(tb)
    return 2 * precision * recall / (precision + recall)


def score_pico(extracted, expected):
    """Score a single PICO extraction against ground truth.
    
    Returns dict with per-element jaccard, f1, and soft match scores.
    """
    scores = {}
    for el in PICO_ELEMENTS:
        ext_val = getattr(extracted, el, None) or ""
        exp_val = expected.get(el) or ""
        j = jaccard(ext_val, exp_val)
        f = token_f1(ext_val, exp_val)
        scores[el] = {
            "jaccard": round(j, 2),
            "f1": round(f, 2),
            "soft_match": j > 0.5,  # keep jaccard as primary soft match
        }
    return scores


# Sanity checks
print(f"\nStemming sanity check:")
print(f"  jaccard('prevents COVID-19 infection', 'prevention of COVID-19 infection') = "
      f"{jaccard('prevents COVID-19 infection', 'prevention of COVID-19 infection'):.2f}")
print(f"  token_f1('prevents COVID-19 infection', 'prevention of COVID-19 infection') = "
      f"{token_f1('prevents COVID-19 infection', 'prevention of COVID-19 infection'):.2f}")
print(f"  jaccard('reduces cancer risk', 'reduced cancer risk') = "
      f"{jaccard('reduces cancer risk', 'reduced cancer risk'):.2f}")
print(f"  token_f1('reduces cancer risk', 'reduced cancer risk') = "
      f"{token_f1('reduces cancer risk', 'reduced cancer risk'):.2f}")

Loaded 30 claims with PICO ground truth
Sample: Intermittent fasting reverses Type 2 diabetes
  Expected: {'population': 'patients with Type 2 diabetes', 'intervention': 'intermittent fasting', 'comparison': None, 'outcome': 'reversal of Type 2 diabetes'}

Stemming sanity check:
  jaccard('prevents COVID-19 infection', 'prevention of COVID-19 infection') = 0.75
  token_f1('prevents COVID-19 infection', 'prevention of COVID-19 infection') = 0.86
  jaccard('reduces cancer risk', 'reduced cancer risk') = 1.00
  token_f1('reduces cancer risk', 'reduced cancer risk') = 1.00


### 10a. Rule-Based PICO (no API key needed)

In [16]:
# Run rule-based PICO extraction on all claims
rule_results = []

for item in pico_gt:
    claim = item["claim"]
    expected = item["pico"]
    entities = extract_entities(claim)
    extracted = extract_pico_rule_based(claim, entities)
    scores = score_pico(extracted, expected)

    rule_results.append({
        "claim": claim,
        "expected": expected,
        "extracted": {
            "population": extracted.population,
            "intervention": extracted.intervention,
            "comparison": extracted.comparison,
            "outcome": extracted.outcome,
        },
        "scores": scores,
    })

# Show detailed results for each claim
print("Rule-Based PICO Extraction Results")
print("=" * 90)
for r in rule_results:
    print(f"\nClaim: {r['claim']}")
    for el in ["population", "intervention", "comparison", "outcome"]:
        exp = r["expected"].get(el) or "(null)"
        ext = r["extracted"].get(el) or "(null)"
        j = r["scores"][el]["jaccard"]
        match = "Y" if r["scores"][el]["soft_match"] else "N"
        print(f"  {el[0].upper()}: {ext:40s}  expect: {exp:35s}  j={j:.2f} [{match}]")

Rule-Based PICO Extraction Results

Claim: Intermittent fasting reverses Type 2 diabetes
  P: (null)                                    expect: patients with Type 2 diabetes        j=0.00 [N]
  I: Intermittent fasting                      expect: intermittent fasting                 j=1.00 [Y]
  C: (null)                                    expect: (null)                               j=1.00 [Y]
  O: Type 2 diabetes                           expect: reversal of Type 2 diabetes          j=0.60 [Y]

Claim: Vitamin D prevents COVID-19 infection
  P: (null)                                    expect: general population                   j=0.00 [N]
  I: Vitamin D                                 expect: vitamin D                            j=1.00 [Y]
  C: (null)                                    expect: (null)                               j=1.00 [Y]
  O: COVID-19 infection                        expect: prevention of COVID-19 infection     j=0.50 [N]

Claim: Metformin reduces cancer risk in 

### 10b. LLM PICO (needs ANTHROPIC_API_KEY)

In [17]:
import time

# Run LLM PICO extraction on all claims
llm_results = []

for item in pico_gt:
    claim = item["claim"]
    expected = item["pico"]
    entities = extract_entities(claim)

    try:
        extracted = extract_pico_with_llm(claim, entities)
        time.sleep(0.5)  # rate limit
    except Exception as e:
        print(f"  LLM failed for: {claim[:50]}... ({e})")
        extracted = PICO()

    scores = score_pico(extracted, expected)

    llm_results.append({
        "claim": claim,
        "expected": expected,
        "extracted": {
            "population": extracted.population,
            "intervention": extracted.intervention,
            "comparison": extracted.comparison,
            "outcome": extracted.outcome,
        },
        "scores": scores,
    })

# Show detailed results
print("LLM PICO Extraction Results")
print("=" * 90)
for r in llm_results:
    print(f"\nClaim: {r['claim']}")
    for el in ["population", "intervention", "comparison", "outcome"]:
        exp = r["expected"].get(el) or "(null)"
        ext = r["extracted"].get(el) or "(null)"
        j = r["scores"][el]["jaccard"]
        match = "Y" if r["scores"][el]["soft_match"] else "N"
        print(f"  {el[0].upper()}: {ext:40s}  expect: {exp:35s}  j={j:.2f} [{match}]")

LLM PICO Extraction Results

Claim: Intermittent fasting reverses Type 2 diabetes
  P: people with Type 2 diabetes               expect: patients with Type 2 diabetes        j=0.67 [Y]
  I: intermittent fasting                      expect: intermittent fasting                 j=1.00 [Y]
  C: (null)                                    expect: (null)                               j=1.00 [Y]
  O: reverses Type 2 diabetes                  expect: reversal of Type 2 diabetes          j=0.80 [Y]

Claim: Vitamin D prevents COVID-19 infection
  P: (null)                                    expect: general population                   j=0.00 [N]
  I: Vitamin D                                 expect: vitamin D                            j=1.00 [Y]
  C: (null)                                    expect: (null)                               j=1.00 [Y]
  O: prevents COVID-19 infection               expect: prevention of COVID-19 infection     j=0.75 [Y]

Claim: Metformin reduces cancer risk in diabeti

### 10b½. LLM-as-Judge Scoring

Use Claude to judge semantic equivalence between extracted and expected PICO elements.
This handles synonyms that token overlap can't — e.g., "effectiveness" vs "improvement".

Batches both rule-based and LLM extractions into a single API call per claim (30 calls total).

In [18]:
import anthropic
import time
from src.config import ANTHROPIC_API_KEY, CLAUDE_MODEL

_JUDGE_SYSTEM = """\
You are evaluating PICO extraction quality. For each element, judge whether the \
extracted value is semantically equivalent to the expected value.

Rules:
- "Y" if they capture the same meaning, even with different wording \
  (e.g., "prevents cancer" ≈ "prevention of cancer", "effective" ≈ "improvement")
- "Y" if both are null/empty
- "N" if one is null and the other is not
- "N" if they refer to different concepts

Respond with ONLY a JSON object, no other text."""


def llm_judge_claim(claim, rule_ext, llm_ext, expected):
    """Judge one claim's PICO extractions against ground truth."""
    user_msg = f'Claim: "{claim}"\n\n'

    for method, ext in [("rule", rule_ext), ("llm", llm_ext)]:
        user_msg += f"{method} extraction:\n"
        for el in PICO_ELEMENTS:
            ext_val = ext.get(el) or "(null)"
            exp_val = expected.get(el) or "(null)"
            user_msg += f'  {el[0].upper()}: extracted="{ext_val}"  expected="{exp_val}"\n'
        user_msg += "\n"

    user_msg += ('Respond with: {"rule": {"P": "Y/N", "I": "Y/N", "C": "Y/N", "O": "Y/N"}, '
                 '"llm": {"P": "Y/N", "I": "Y/N", "C": "Y/N", "O": "Y/N"}}')

    client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)
    msg = client.messages.create(
        model=CLAUDE_MODEL,
        max_tokens=200,
        system=_JUDGE_SYSTEM,
        messages=[{"role": "user", "content": user_msg}],
    )

    text = msg.content[0].text.strip()
    if text.startswith("```"):
        text = text.split("```")[1]
        if text.startswith("json"):
            text = text[4:]
        text = text.strip()

    return json.loads(text)


# Run LLM judge on all 30 claims
judge_results = []
el_map = {"P": "population", "I": "intervention", "C": "comparison", "O": "outcome"}

print("Running LLM-as-Judge on 30 claims...")
for i, (rule_r, llm_r) in enumerate(zip(rule_results, llm_results)):
    try:
        verdict = llm_judge_claim(
            rule_r["claim"], rule_r["extracted"], llm_r["extracted"], rule_r["expected"]
        )
        judge_results.append(verdict)
        time.sleep(0.3)
    except Exception as e:
        print(f"  Failed on claim {i}: {e}")
        judge_results.append({"rule": {"P": "N", "I": "N", "C": "N", "O": "N"},
                              "llm":  {"P": "N", "I": "N", "C": "N", "O": "N"}})

    if (i + 1) % 10 == 0:
        print(f"  {i + 1}/30 done")

print(f"Done. {len(judge_results)} claims judged.")

# Store judge verdicts back into results for the aggregate
for i, jv in enumerate(judge_results):
    for el_short, el_full in el_map.items():
        rule_results[i]["scores"][el_full]["judge"] = jv["rule"][el_short] == "Y"
        llm_results[i]["scores"][el_full]["judge"] = jv["llm"][el_short] == "Y"

Running LLM-as-Judge on 30 claims...
  10/30 done
  Failed on claim 13: Expecting ':' delimiter: line 1 column 93 (char 92)
  20/30 done
  30/30 done
Done. 30 claims judged.


### 10c. Rule-Based vs LLM — Aggregate Comparison

In [19]:
import pandas as pd

n = len(pico_gt)
total = n * 4

# --- Build aggregate rows for all three metrics ---
metrics = ["jaccard", "f1", "judge"]
metric_labels = {"jaccard": "Jaccard (stemmed)", "f1": "Token F1 (stemmed)", "judge": "LLM Judge"}

print("PICO Extraction: Rule-Based vs LLM — 3 Metrics")
print("=" * 90)

for metric in metrics:
    print(f"\n  {metric_labels[metric]}")
    print(f"  {'elem':5s}  {'rule':>8s}  {'llm':>8s}  {'winner':>8s}")
    print(f"  {'-' * 35}")

    rule_total, llm_total = 0, 0
    for el in PICO_ELEMENTS:
        if metric == "judge":
            rule_acc = sum(1 for r in rule_results if r["scores"][el].get("judge", False)) / n
            llm_acc  = sum(1 for r in llm_results  if r["scores"][el].get("judge", False)) / n
        else:
            rule_acc = sum(r["scores"][el][metric] for r in rule_results) / n
            llm_acc  = sum(r["scores"][el][metric] for r in llm_results) / n

        rule_total += rule_acc
        llm_total  += llm_acc
        winner = "<<" if llm_acc > rule_acc + 0.02 else (">>" if rule_acc > llm_acc + 0.02 else "  ")

        if metric == "judge":
            print(f"  {el[0].upper():5s}  {rule_acc:7.0%}   {llm_acc:7.0%}    {winner}")
        else:
            print(f"  {el[0].upper():5s}  {rule_acc:8.2f}  {llm_acc:8.2f}  {winner:>8s}")

    rule_avg = rule_total / 4
    llm_avg  = llm_total / 4
    winner = "<<" if llm_avg > rule_avg + 0.02 else (">>" if rule_avg > llm_avg + 0.02 else "  ")

    if metric == "judge":
        print(f"  {'ALL':5s}  {rule_avg:7.0%}   {llm_avg:7.0%}    {winner}")
    else:
        print(f"  {'ALL':5s}  {rule_avg:8.2f}  {llm_avg:8.2f}  {winner:>8s}")

# --- Summary comparison ---
print("\n" + "=" * 90)
print("Summary: Overall Accuracy by Metric")
print(f"  {'metric':25s}  {'rule':>8s}  {'llm':>8s}  {'winner':>8s}")
print(f"  {'-' * 55}")

for metric in metrics:
    if metric == "judge":
        rule_acc = sum(1 for r in rule_results for el in PICO_ELEMENTS
                       if r["scores"][el].get("judge", False)) / total
        llm_acc  = sum(1 for r in llm_results for el in PICO_ELEMENTS
                       if r["scores"][el].get("judge", False)) / total
        rule_s, llm_s = f"{rule_acc:.0%}", f"{llm_acc:.0%}"
    elif metric == "jaccard":
        rule_acc = sum(1 for r in rule_results for el in PICO_ELEMENTS
                       if r["scores"][el]["jaccard"] > 0.5) / total
        llm_acc  = sum(1 for r in llm_results for el in PICO_ELEMENTS
                       if r["scores"][el]["jaccard"] > 0.5) / total
        rule_s, llm_s = f"{rule_acc:.0%}", f"{llm_acc:.0%}"
    else:  # f1
        rule_acc = sum(1 for r in rule_results for el in PICO_ELEMENTS
                       if r["scores"][el]["f1"] > 0.5) / total
        llm_acc  = sum(1 for r in llm_results for el in PICO_ELEMENTS
                       if r["scores"][el]["f1"] > 0.5) / total
        rule_s, llm_s = f"{rule_acc:.0%}", f"{llm_acc:.0%}"

    winner = "<<" if llm_acc > rule_acc + 0.02 else (">>" if rule_acc > llm_acc + 0.02 else "  ")
    print(f"  {metric_labels[metric]:25s}  {rule_s:>8s}  {llm_s:>8s}  {winner:>8s}")

print(f"\n  n = {n} claims × 4 elements = {total} judgments")
print(f"  >> = rule-based wins, << = LLM wins")

PICO Extraction: Rule-Based vs LLM — 3 Metrics

  Jaccard (stemmed)
  elem       rule       llm    winner
  -----------------------------------
  P          0.20      0.60        <<
  I          0.69      0.98        <<
  C          0.90      0.97        <<
  O          0.35      0.65        <<
  ALL        0.53      0.80        <<

  Token F1 (stemmed)
  elem       rule       llm    winner
  -----------------------------------
  P          0.20      0.64        <<
  I          0.70      0.99        <<
  C          0.90      0.97        <<
  O          0.44      0.71        <<
  ALL        0.56      0.83        <<

  LLM Judge
  elem       rule       llm    winner
  -----------------------------------
  P          20%       70%    <<
  I          67%       93%    <<
  C          90%       93%    <<
  O          50%       93%    <<
  ALL        57%       88%    <<

Summary: Overall Accuracy by Metric
  metric                         rule       llm    winner
  ---------------------------

In [22]:
# --- Per-claim breakdown: which claims does each method struggle with? ---

print("Per-Claim Soft Match Breakdown")
print("=" * 100)
print(f"{'claim':50s}  {'rule P I C O':^13s}  {'llm  P I C O':^13s}  {'rule':>4s}  {'llm':>4s}")
print("-" * 100)

rule_totals, llm_totals = 0, 0

for rule_r, llm_r in zip(rule_results, llm_results):
    claim_short = rule_r["claim"][:48]

    rule_flags = "".join(
        "Y" if rule_r["scores"][el]["soft_match"] else "." for el in PICO_ELEMENTS
    )
    llm_flags = "".join(
        "Y" if llm_r["scores"][el]["soft_match"] else "." for el in PICO_ELEMENTS
    )

    rule_n = sum(1 for el in PICO_ELEMENTS if rule_r["scores"][el]["soft_match"])
    llm_n  = sum(1 for el in PICO_ELEMENTS if llm_r["scores"][el]["soft_match"])
    rule_totals += rule_n
    llm_totals  += llm_n

    # Highlight rows where methods disagree
    marker = ""
    if rule_n > llm_n:
        marker = "  rule+"
    elif llm_n > rule_n:
        marker = "  llm+"

    # Space out the PICO flags for readability
    rule_spaced = " ".join(rule_flags)
    llm_spaced  = " ".join(llm_flags)
    print(f"  {claim_short:48s}  {rule_spaced:>13s}  {llm_spaced:>13s}  {rule_n}/4   {llm_n}/4{marker}")

print("-" * 100)
max_total = len(pico_gt) * 4
print(f"  {'TOTAL':48s}  {'':13s}  {'':13s}  {rule_totals}/{max_total}  {llm_totals}/{max_total}")

# Claims where rule-based fails but LLM succeeds (and vice versa)
print("\n\nClaims where methods disagree:")
print("-" * 70)
for rule_r, llm_r in zip(rule_results, llm_results):
    for el in PICO_ELEMENTS:
        rule_ok = rule_r["scores"][el]["soft_match"]
        llm_ok  = llm_r["scores"][el]["soft_match"]
        if rule_ok != llm_ok:
            winner = "LLM" if llm_ok else "Rule"
            print(f"  [{winner:4s} wins {el[0].upper()}]  {rule_r['claim'][:55]}")
            print(f"    rule: {rule_r['extracted'].get(el) or '(null)'}")
            print(f"    llm:  {llm_r['extracted'].get(el) or '(null)'}")
            print(f"    gold: {rule_r['expected'].get(el) or '(null)'}")
            print()

Per-Claim Soft Match Breakdown
claim                                               rule P I C O   llm  P I C O   rule   llm
----------------------------------------------------------------------------------------------------
  Intermittent fasting reverses Type 2 diabetes           . Y Y Y        Y Y Y Y  3/4   4/4  llm+
  Vitamin D prevents COVID-19 infection                   . Y Y .        . Y Y Y  2/4   3/4  llm+
  Metformin reduces cancer risk in diabetic patien        Y Y Y Y        Y Y Y Y  4/4   4/4
  Turmeric is as effective as ibuprofen for arthri        . Y Y Y        . Y Y .  3/4   2/4  rule+
  Drinking 8 glasses of water a day prevents kidne        . Y Y .        . Y Y Y  2/4   3/4  llm+
  Aspirin interacts with warfarin and increases bl        . . Y Y        Y Y Y Y  2/4   4/4  llm+
  Patients with hypertension should take ACE inhib        . . Y .        Y . Y .  1/4   2/4  llm+
  Omega-3 supplements reduce triglycerides by 30%         . . Y .        . Y Y .  1/4   2/4  l

---
## 11. Summary of Findings

### 11a. Retrieval Planner — Method Discrimination

The Retrieval Planner is a ReAct agent that decides which retrieval methods to use per sub-claim. We tested it on 8 diverse health claims covering treatment efficacy, drug interactions, recommendations, and quantitative claims.

**Key results:**
- **4/4 discrimination checks pass** — the agent correctly restricts optional methods to relevant claim types
- `drugbank_api` fires **only** for drug interaction claims (aspirin + warfarin)
- `deep_search` fires **only** for quantitative claims (30% triglyceride reduction)
- `guideline_store` fires selectively for guideline-relevant topics (hypertension, diabetes)
- 2-3 claims correctly receive **only the 3 baseline methods** (pubmed, semantic scholar, cross-encoder)
- Average methods per claim: **4.1** (selective, not exhaustive)

**System prompt tuning (v1 → v2):**
- Tightened the prompt to emphasize selectivity: "Be selective, not comprehensive"
- Added explicit rules: only add `cochrane_api`/`clinical_trials` for treatment claims, only `drugbank_api` for interaction claims
- Result: optional method usage dropped (clinical_trials 75%→50%, cochrane_api 50%→25%) while discrimination checks stayed at 4/4

### 11b. PICO Extraction — Rule-Based vs LLM

Evaluated both extraction methods against 30 hand-labeled health claims using 3 complementary metrics.

**Overall accuracy (% of 120 element extractions judged correct):**

| Metric | Rule-Based | LLM | Gap |
|--------|-----------|-----|-----|
| Jaccard (stemmed, >0.5) | 51% | 78% | +27% |
| Token F1 (stemmed, >0.5) | 58% | 84% | +26% |
| LLM Judge (semantic) | 57% | **88%** | +31% |

**Per-element accuracy (LLM Judge):**

| Element | Rule-Based | LLM | Notes |
|---------|-----------|-----|-------|
| Population | 20% | 70% | Rule-based rarely identifies populations |
| Intervention | 67% | 93% | LLM handles diverse intervention phrasings |
| Comparison | 90% | 93% | Both strong — nulls are easy to match |
| Outcome | 50% | 93% | Biggest gap — LLM understands outcome semantics |

**Why 3 metrics?**
- **Jaccard** (|A∩B| / |A∪B|): Simple token overlap. Penalizes extra/missing words equally.
- **Token F1** (harmonic mean of precision and recall): Like SQuAD scoring. Separates "extracted too much" from "extracted too little".
- **LLM Judge** (Claude semantic equivalence): Handles synonyms ("effectiveness" ≈ "improvement", "prevents" ≈ "prevention") that token metrics miss. Most accurate but costs API calls.

Token metrics undercount LLM accuracy because of verb/noun form differences (e.g., "prevents cancer" vs "prevention of cancer" share only 1 of 4 stemmed tokens). The LLM Judge correctly recognizes these as equivalent.

### 11c. Prompt Fix — Comparison Hallucination

**Problem found:** The PICO extraction prompt told the LLM to *infer* comparators when none were stated, producing hallucinated values like "no treatment", "standard care", "no vitamin D".

**Fix:** Changed the C (Comparison) instruction from:
> "If not stated, infer the most reasonable comparator (e.g., 'no treatment', 'standard care', 'placebo')"

To:
> "ONLY if an explicit comparator is stated in the claim (e.g., 'more effective than X', 'as good as Y'). If no comparison is mentioned, use null."

**Result:** LLM comparison accuracy jumped from **20% → 93%** (LLM Judge).

### 11d. Scoring Methodology Notes

- **Stemming** (Snowball stemmer) applied to all token-based metrics to handle morphological variants ("reduces"→"reduc", "prevention"→"prevent")
- **Soft match threshold** of >0.5 for Jaccard and F1 — requires more than half token overlap
- **Ground truth** is 30 hand-labeled claims covering: treatment efficacy (8), drug interactions (2), lifestyle interventions (5), supplements (6), safety/recommendations (4), comparisons (5)
- **LLM Judge** batches both methods' extractions into a single API call per claim (30 calls total for both methods)